In [2]:
import tensorflow as tf

In [10]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x * x * y + y + 2

In [3]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()
print(result)

42


In [4]:
graph = tf.Graph()

In [5]:
with graph.as_default():
    x2 = tf.Variable(2, name='x2')
    

In [6]:
x2.graph is graph

True

In [7]:
x2.graph is tf.get_default_graph()

False

In [8]:
tf.reset_default_graph()

In [9]:
x is tf.get_default_graph()

False

In [12]:
w = tf.constant(3)
x = w + 2
y = w + 5
z = x * 3

In [13]:
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

8
15


In [14]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])

In [15]:
y_val, z_val

(8, 15)

In [16]:
# Linear Regression with TensorFlow

In [3]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [4]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

In [5]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
with tf.Session() as sess:
    theta_value = theta.eval()

In [6]:
theta_value

array([[-3.7465141e+01],
       [ 4.3573415e-01],
       [ 9.3382923e-03],
       [-1.0662201e-01],
       [ 6.4410698e-01],
       [-4.2513184e-06],
       [-3.7732250e-03],
       [-4.2664889e-01],
       [-4.4051403e-01]], dtype=float32)

In [7]:
n_epochs = 1000
learning_rate = 0.01

In [36]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [37]:
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)


In [38]:
init = tf.global_variables_initializer()

In [39]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, 'MSE=', mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE= 7.3535733


Epoch 100 MSE= 0.7052758


Epoch 200 MSE= 0.5685586


Epoch 300 MSE= 0.5536344


Epoch 400 MSE= 0.5453737
Epoch 500 MSE= 0.5395295


Epoch 600 MSE= 0.53531677


Epoch 700 MSE= 0.5322741
Epoch 800 MSE= 0.53007615


Epoch 900 MSE= 0.52848685


In [40]:
best_theta

array([[ 2.0685523e+00],
       [ 7.8661549e-01],
       [ 1.3206096e-01],
       [-1.4430261e-01],
       [ 1.8787360e-01],
       [ 8.9631026e-04],
       [-3.9535053e-02],
       [-8.4323984e-01],
       [-8.0695605e-01]], dtype=float32)

In [30]:
# Feeding Data to the Training Algorithm

In [31]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

In [32]:
print(B_val_1)

[[6. 7. 8.]]


In [33]:
print(B_val_2)

[[ 9. 10. 11.]
 [12. 13. 14.]]


In [46]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')


In [47]:
theta = tf.Variable(tf.random_uniform([n+1, 1], -1., 1., seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [35]:
batch_size = 100
n_batch = int(np.ceil(m / batch_size))

In [50]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batch + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

In [51]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batch):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()

In [52]:
best_theta

array([[ 2.0714476 ],
       [ 0.8462012 ],
       [ 0.11558535],
       [-0.26835832],
       [ 0.32982782],
       [ 0.00608358],
       [ 0.07052915],
       [-0.87988573],
       [-0.8634251 ]], dtype=float32)

In [54]:
# Saving and Restoring Models
n_epochs = 1000                                                                       # not shown in the book
learning_rate = 0.01                                                                  # not shown

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")            # not shown
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")            # not shown
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")                                      # not shown
error = y_pred - y                                                                    # not shown
mse = tf.reduce_mean(tf.square(error), name="mse")                                    # not shown
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            # not shown
training_op = optimizer.minimize(mse)                                                 # not shown
init = tf.global_variables_initializer()
Saver = tf.train.Saver()

In [56]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch:', epoch, "MSE:", mse.eval())
            save_path = Saver.save(sess, "/tmp/my_mode.ckpt")
        sess.run(training_op)
    best_theta = theta.eval()
    save_path = Saver.save(sess, "/tmp/my_model_final.ckpt")


Epoch: 0 MSE: 2.7544272


Epoch: 100 MSE: 0.63222194


Epoch: 200 MSE: 0.5727796


Epoch: 300 MSE: 0.5585005


Epoch: 400 MSE: 0.54906934


Epoch: 500 MSE: 0.5422877


Epoch: 600 MSE: 0.5373788


Epoch: 700 MSE: 0.5338219


Epoch: 800 MSE: 0.5312427


Epoch: 900 MSE: 0.52937055


In [57]:
with tf.Session() as sess:
    Saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [58]:
best_theta_restored

array([[ 2.06855249e+00],
       [ 7.74078071e-01],
       [ 1.31192386e-01],
       [-1.17845066e-01],
       [ 1.64778143e-01],
       [ 7.44078017e-04],
       [-3.91945094e-02],
       [-8.61356676e-01],
       [-8.23479772e-01]], dtype=float32)

In [59]:
tf.reset_default_graph()

In [60]:
saver = tf.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")
theta = tf.get_default_graph().get_tensor_by_name("theta:0")
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")  # this restores the graph's state
    best_theta_restored = theta.eval() 

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [62]:
best_theta_restored


array([[ 0.9045429 ],
       [ 0.35481548],
       [ 0.5906365 ],
       [ 0.51156354],
       [-0.04808879],
       [ 0.26202965],
       [-0.62795925],
       [-0.7713845 ],
       [-0.32755637]], dtype=float32)

In [63]:
from datetime import datetime

In [83]:
now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = '{}/run-{}/'.format(root_logdir, now)

In [84]:
logdir

'tf_logs/run-20180427142552/'

In [85]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [67]:
n_batch

207

In [86]:
n_epoches = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [87]:
with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown

    for epoch in range(n_epoches):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()  

In [76]:
file_writer.close()

In [88]:
best_theta

array([[ 2.070016  ],
       [ 0.8204561 ],
       [ 0.1173173 ],
       [-0.22739051],
       [ 0.3113402 ],
       [ 0.00353193],
       [-0.01126994],
       [-0.91643935],
       [-0.8795008 ]], dtype=float32)

In [82]:
import os
os.getcwd()

'/Users/Conan/Documents/codeForFun/codeForFun'

In [93]:
# Name Scopes

In [94]:
with tf.name_scope('loss') as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name='mse')
    

In [97]:
print(error.op.name)
print(mse.op.name)

loss/sub
loss/mse


In [98]:
# Rectified linear unit
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')

w1 = tf.Variable(tf.random_normal((n_features, 1)), name='weight1')
w2 = tf.Variable(tf.random_normal((n_features, 1), name='weight2'))

b1 = tf.Variable(0.0, name='bias1')
b2 = tf.Variable(0.0, name='bias2')

z1 = tf.add(tf.matmul(X, w1), b1, name='z1')
z2 = tf.add(tf.matmul(X, w2), b2, name='z2')

relu1 = tf.maximum(z1, 0., name='relu1')
relu2 = tf.maximum(z2, 0., name='relu2')

output = tf.add(relu1, relu2, name='output')